## Variational Autoencoder

In [1]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [2]:
if not os.path.exists('save/var_ae'):
    os.mkdir('save/var_ae')

In [3]:
def to_img(x):
    x = x.clamp(0, 1)
    x = x.reshape(x.size(0), 1, 28, 28)
    return x

In [4]:
num_epoches = 100
batch_size = 128
learning_rate = 1e-3
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu") # whether GPU is supportted

In [5]:
img_transform = transforms.Compose([
    transforms.ToTensor()
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [6]:
dataset = datasets.MNIST('../_data/mnist', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [7]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        eps = torch.FloatTensor(std.size()).normal_().to(device)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar

In [8]:
model = VAE().to(device)
reconstruction_function = nn.MSELoss(size_average=False)

In [9]:
def loss_function(recon_x, x, mu, logvar):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    BCE = reconstruction_function(recon_x, x)  # mse loss
    # loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    # KL divergence
    return BCE + KLD

In [10]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [11]:
for epoch in range(num_epoches):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(dataloader):
        img, _ = data
        img = img.reshape(img.size(0), -1)
        img = img.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(img)
        loss = loss_function(recon_batch, img, mu, logvar)
        loss.backward()
        train_loss += loss
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch+1} [{batch_idx*len(img)}/{len(dataset)} ({100.*batch_idx/len(dataloader):.0f}%)]\tLoss: {loss/len(img):.6f}')

    print(f'====> Epoch: {epoch+1} Average loss: {train_loss/len(dataset):.4f}')
    if epoch % 10 == 0:
        save = to_img(recon_batch)
        save_image(save, 'save/var_ae/image_{}.png'.format(epoch))

Train Epoch: 1 [0/60000 (0%)]	Loss: 183.171616
Train Epoch: 1 [12800/60000 (21%)]	Loss: 45.495533
Train Epoch: 1 [25600/60000 (43%)]	Loss: 42.757023
Train Epoch: 1 [38400/60000 (64%)]	Loss: 39.825748
Train Epoch: 1 [51200/60000 (85%)]	Loss: 39.333122
====> Epoch: 1 Average loss: 45.1352
Train Epoch: 2 [0/60000 (0%)]	Loss: 34.071899
Train Epoch: 2 [12800/60000 (21%)]	Loss: 35.072456
Train Epoch: 2 [25600/60000 (43%)]	Loss: 35.176964
Train Epoch: 2 [38400/60000 (64%)]	Loss: 32.953926
Train Epoch: 2 [51200/60000 (85%)]	Loss: 35.158745
====> Epoch: 2 Average loss: 35.0990
Train Epoch: 3 [0/60000 (0%)]	Loss: 32.176605
Train Epoch: 3 [12800/60000 (21%)]	Loss: 33.827629
Train Epoch: 3 [25600/60000 (43%)]	Loss: 33.355484
Train Epoch: 3 [38400/60000 (64%)]	Loss: 32.085175
Train Epoch: 3 [51200/60000 (85%)]	Loss: 32.856995
====> Epoch: 3 Average loss: 33.2810
Train Epoch: 4 [0/60000 (0%)]	Loss: 30.144249
Train Epoch: 4 [12800/60000 (21%)]	Loss: 32.276543
Train Epoch: 4 [25600/60000 (43%)]	Loss: 

In [12]:
torch.save(model.state_dict(), 'save/var_ae/vae.pytorch')